 # Time series models on example of TESLA stock prices

Dependencies:
    - pandas
    - pandas-datareader
    - numpy
    - matplotlib
    - scikit-learn
    - seaborn
    - datetime
    - warnings
    - sys
    - itertools
    - statsmodels
    - pyramid-arima (pip install pyramid-arima)
    - keras
    - pykalman 
    - bokeh

<img src="assets/img/tesla.jpg" alt="drawing" width="400"/>

The goal of the excercise is to create a time-series model that tracks close enough to the closing price of Tesla stocks each day. Tesla is a fascinating case for the analysis because it has been the most valuable car manufacturing company in the USA in 2017 despite selling only 4 car models. Besides, Tesla has never had a profitable year, so it's stock price is speculative. 

## ARIMA model
First, we can see if ARIMA models are capable of capturing such turbulent series. We'll follow the standard path:
+ Visualize the time series
+ Stationarize the series
+ Plot ACF/PACF charts and find optimal parameters
+ Build the ARIMA model
+ Make predictions

### 1. Data exploration

In [4]:
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import warnings
warnings.simplefilter('ignore')
import sys
import itertools
import statsmodels.api as sm
from pyramid.arima import auto_arima
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

plt.style.use('fivethirtyeight')
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

The tsla dataset contains daily data on TSLA stocks from 2013-8-15 to 2018-8-15. We're interested in Close price series.

In [ ]:
#Import the data
dateparse = lambda date: pd.datetime.strptime(date, '%Y-%m-%d')
tsla = pd.read_csv('assets/data/tsla', parse_dates=['Date'], index_col='Date', date_parser=dateparse,)
tsla.head()

In [ ]:
#plot the daily Close price
tsla.Close.plot(lw=2.5, figsize=(12,5))
plt.show()

As far as we can tell from this plot, the series for Close stock price look non stationary, with no obvious linear trend or seasonality. The distribution is bimodal (with two peaks).

In [ ]:
#distribution plot of tsla series
tsla_close = pd.Series.to_frame(tsla.Close)
plt.figure(figsize=(10,5))
sns.distplot(tsla_close.dropna(), color='g')
plt.show()

Let's now try to decompose the data into trend and seasonal components. By changing frequency, we can check for the presence weekly, monthly or yearly seasonal fluctuations.  

In [ ]:
#decompose into seasonal and trend components
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(tsla_close, freq=30, model='additive')  
fig = plt.figure()  
fig = decomposition.plot()  
fig.set_size_inches(15, 8)

#### Smoothing out data with moving average
Rather than calculating the average on the whole dataset, moving average (also called rolling mean) calculates the average of a subset with a certain window size, and shifts forward. Moving average is used to smooth out short-term fluctuations and highlight longer-term trends or cycles.
Let's see how moving average works by taking a shorter recent time span.

In [ ]:
tsla_recent = tsla_close.loc['2018-5-15':'2018-8-15']
rroll_d3 = tsla_recent.rolling(window=3).mean()
rroll_d7 = tsla_recent.rolling(window=7).mean()
rroll_d14 = tsla_recent.rolling(window=14).mean()

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(tsla_recent.index, tsla_recent, lw=3, alpha=0.8,label='Original observations')
plt.plot(tsla_recent.index, rroll_d3, lw=3, alpha=0.8,label='Rolling mean (window 3)')
plt.plot(tsla_recent.index, rroll_d7, lw=3, alpha=0.8,label='Rolling mean (window 7)')
plt.plot(tsla_recent.index, rroll_d14, lw=3, alpha=0.8,label='Rolling mean (window 14)')
plt.title('Tesla Close Price 2013-5-15 to 2018-8-15')
plt.tick_params(labelsize=12)
plt.legend(loc='upper left', fontsize=12)
plt.show()

### 2. Test for stationarity and stationarize the data
The Augmented Dicky Fuller test is a type of statistical test called a unit root test.
The intuition behind a unit root test is that it determines how strongly a time series is defined by a trend.
There are no. of unit root tests and ADF is one of the most widely used
1. Null Hypothesis (H0): Null hypothesis of the test is that the time series can be represented by a unit root that is not stationary.
2. Alternative Hypothesis (H1): Alternative Hypothesis of the test is that the time series is stationary.
Interpretation of p value
1. p value > 0.05: Accepts the Null Hypothesis (H0), the data has a unit root and is non-stationary.
2. p value < = 0.05: Rejects the Null Hypothesis (H0), the data is stationary.

In [ ]:
# adf test for stationarity
from statsmodels.tsa.stattools import adfuller

def test_stationarity(x):
#Determing rolling statistics
    rolmean = x.rolling(window=22,center=False).mean()
    rolstd = x.rolling(window=12,center=False).std()
    
    #Plot rolling statistics:
    orig = plt.plot(x, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)

#Perform Dickey Fuller test    
    result=adfuller(x)
    print('ADF Stastistic: %f'%result[0])
    print('p-value: %f'%result[1])
    pvalue=result[1]
    for key,value in result[4].items():
         if result[0]>value:
            print("The graph is non stationary")
            break
         else:
            print("The graph is stationary")
            break;
    print('Critical values:')
    for key,value in result[4].items():
        print('\t%s: %.3f ' % (key, value))
ts = tsla.Close
test_stationarity(ts)

Since p value is greater than 0.05 the time series is non stationary.
Let's try transforming the data.
#### Log transformation of the series
is typically used to unskew highly skewed data, thus helping in forecasting process.

In [ ]:
#log transform series and test stationarity
ts_log = np.log(ts)
plt.plot(ts_log, color = "green")
plt.show()
test_stationarity(ts_log)

The series remain non stationary as p-value is still greater than 0.05 and some further transformations can be applied. Let's try to go ahead and use differencing. The log differencing of the series has the meaning. We transform price levels into the rate of returns. The reason for multiplying by 100 is due to numerical problems in the estimation
part. This will not affect the structure of the model since it is just a linear scaling.

In [ ]:
#take first differences and test stationarity
ts_log_diff = (ts_log - ts_log.shift())*100
plt.plot(ts_log_diff, color = "green")
plt.show()
ts_log_diff.head()

In [ ]:
#test for stationarity
ts_log_diff.dropna(inplace=True)
test_stationarity(ts_log_diff)

In [ ]:
#distribution plot
plt.figure(figsize=(10,5))
sns.distplot(ts_log_diff.dropna(), color='y')
plt.show()

The time series is now stationary as p value is less than 0.05. The distributions of stock returns (log difference of stock prices) is close to normal, with mean zero. We can now proceed with fitting ARIMA models.

### 3. Plot and inspect the empirical autocorrelation function
As we remember from the lecture, the ACF of a stationary AR process of order p goes to zero at an exponential rate, while the PACF becomes zero after lag p. For an MA process of order q the theoretical ACF and PACF show the reverse behaviour, the ACF truncates after lag q and the PACF goes to zero at an exponential rate. These properties can be used as a guide to choose the orders of an ARMA model. Here in this data we observe the presence of both AR and MA behaviours. Thus, ARIMA might give the most parsimonius representation.

In [ ]:
#plots of acf/pacf
plt.figure(1, figsize=(12,15))
plt.subplot(211)
plot_acf(ts_log_diff,ax=plt.gca(),lags = range(1, 200))
plt.subplot(212)
plot_pacf(ts_log_diff, ax=plt.gca(), lags = range(1, 200))
plt.show()

We can now use the triplets of parameters defined above to automate the process of training and evaluating ARIMA models on different combinations. In Statistics and Machine Learning, this process is known as grid search (or hyperparameter optimization) for model selection.

When evaluating and comparing statistical models fitted with different parameters, each can be ranked against one another based on how well it fits the data or its ability to accurately predict future data points. We will use the AIC (Akaike Information Criterion) and BIC (Bayesian information criterion) values, which is conveniently returned with ARIMA models fitted using statsmodels. The AIC and BIC measure how well a model fits the data while taking into account the overall complexity of the model. A model that fits the data very well while using lots of features will be assigned a larger AIC/BIC score than a model that uses fewer features to achieve the same goodness-of-fit. Therefore, we are interested in finding the model that yields the lowest AIC/BIC values.

In [ ]:
#fit autoarima
stepwise_fit = auto_arima(ts_log, start_p=0, start_q=0, max_p=20, max_q=10, m=7,
                      start_P=0, seasonal=False, trace=True,
                      error_action='ignore',  # don't want to know if an order does not work
                      suppress_warnings=False,  # don't want convergence warnings
                      stepwise=True, maxiter=200)  # set to stepwise

Minimum AIC is achieved for ARIMA(1,1,1). Let's fit this model.

In [ ]:
#fit arima(1,1,1)
tsla_close.index = pd.DatetimeIndex(ts_log.index.values,
                               freq=tsla_close.index.inferred_freq)
model = ARIMA(np.log(tsla_close), order=(1,1,1))  
results = model.fit(disp=0)
print(results.summary())

In [ ]:
# plot residual errors
residuals = DataFrame(results.resid)
residuals.plot(figsize=(12,5))
plt.figure(figsize=(10,5))
sns.distplot(residuals.dropna(), color='g')
plt.show()

# qq plot
import scipy.stats as stats
stats.probplot(results.resid, dist="norm", plot=plt)
plt.title("Normal Q-Q plot")
plt.show()

print(residuals.describe())

The residuals are drawn from a distribution that is over-dispersed relative to a normal distribution. Over-dispersed data has an increased number of outliers (i.e. the distribution has fatter tails than a normal distribution). Over-dispersed data is also known as having a leptokurtic distribution and as having positive excess kurtosis. On a Q-Q plot over-dispersed data appears as a flipped S shape (the opposite of under-dispersed data).

### 4. Forecast with ARIMA

In [ ]:
# Divide data into train and test
size = int(len(ts_log)-14)
train_arima, test_arima = ts_log[0:size], ts_log[size:len(ts_log)]

In [ ]:
#Make predictions based on ARIMA(1,1,1)
history = [x for x in train_arima]
predictions = list()
originals = list()
error_list = list()

print('Printing Predicted vs Expected Values...')
print('\n')
for t in range(len(test_arima)):
    model = ARIMA(history, order=(1, 1, 1))
    model_fit = model.fit(disp=0)
    
    output = model_fit.forecast()
    
    pred_value = output[0]
    
        
    original_value = test_arima[t]
    history.append(original_value)
    
    pred_value = np.exp(pred_value)
    
    
    original_value = np.exp(original_value)
    
    # Calculating the error
    error = ((abs(pred_value - original_value)) / original_value) * 100
    error_list.append(error)
    print('predicted = %f,   expected = %f,   error = %f ' % (pred_value, original_value, error), '%')
    
    predictions.append(float(pred_value))
    originals.append(float(original_value))

In [ ]:
# After iterating over entire test set the overall mean error is calculated.   
print('\n Mean Error in Predicting Test Case Articles : %f ' % (sum(error_list)/float(len(error_list))), '%')

#Plot 1-day ahead forecast vs the real value
plt.figure(figsize=(8, 6))
test_day = [t
           for t in range(len(test_arima))]
labels={'Orginal','Predicted'}
plt.plot(test_day, predictions, color= 'green')
plt.plot(test_day, originals, color = 'orange')
plt.title('Expected Vs Predicted Views Forecasting')
plt.xlabel('Day')
plt.ylabel('Closing Price')
plt.legend(labels)
plt.show()

<img src="assets/img/musk.jpg" alt="drawing" width="400"/>

The оne period ahead forecast seems to be close to the previous period value. It's because if you inspect the AR(1) coefficient, you can notice that it's close to 0.9871, so essentially the series are very close to the random walk process (the next value equals to the previous value plus some random error).

## Recurrent Neural Network to Predict Stock Prices

What if we run a RNN on the same series?

In [ ]:
# Reading CSV file into training set
training_set = pd.read_csv('assets/data/tsla_train')
training_set.head()

In [ ]:
# Reading CSV file into test set
test_set = pd.read_csv('assets/data/tsla_test')
test_set.head()

In [ ]:
# Getting relevant feature
training_set = training_set.iloc[:,2:3]
training_set.head()

In [ ]:
# Converting to 2D array
training_set = training_set.values
training_set

In [ ]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
training_set = sc.fit_transform(training_set)
training_set

In [ ]:
len(training_set)

In [ ]:
# Getting the inputs and the ouputs
X_train = training_set[0:1043]
y_train = training_set[1:1044]

# Example
today = pd.DataFrame(X_train[0:5])
tomorrow = pd.DataFrame(y_train[0:5])
ex = pd.concat([today, tomorrow], axis=1)
ex.columns = (['today', 'tomorrow'])
ex

In [ ]:
# Reshaping into required shape for Keras
X_train = np.reshape(X_train, (1043, 1, 1))
X_train

In [ ]:
# Initializing the Recurrent Neural Network
regressor = Sequential()

In [ ]:
# Adding the input layer and the LSTM layer
regressor.add(LSTM(units = 4, activation = 'sigmoid', input_shape = (None, 1)))

In [ ]:
# Adding the output layer
regressor.add(Dense(units = 1))

In [ ]:
# Compiling the Recurrent Neural Network
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['accuracy'])

In [ ]:
# Fitting the Recurrent Neural Network to the Training set
history=regressor.fit(X_train, y_train, batch_size = 32, epochs = 200, validation_split=0.33)

In [ ]:
# plot train and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Getting relevant feature# Gettin 
real_stock_price = test_set.iloc[:,2:3]
real_stock_price.head()
#len(real_stock_price)

In [ ]:
# Converting to 2D array
real_stock_price = real_stock_price.values

In [ ]:
# Getting the predicted stock price of 2017
inputs = real_stock_price
inputs = sc.transform(inputs)
inputs = np.reshape(inputs, (261, 1, 1))
predicted_stock_price = regressor.predict(inputs)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
from math import pi
from bokeh.plotting import figure, show, output_notebook
from datetime import date
from datetime import datetime

In [ ]:
%matplotlib notebook
output_notebook()
df = test_set
df.head()
df['date'] = pd.to_datetime(df.Date)

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p = figure(x_axis_type="datetime", tools=TOOLS, plot_width=1000, toolbar_location="left",y_axis_label = "TSLA Price",
          x_axis_label = "Date")

p.line(df.date,predicted_stock_price.flatten(),line_width=1,line_color = 'blue',legend="Real TSLA Stock Price")
p.line(df.date,real_stock_price.flatten(),line_width=1,line_color = 'red',legend="TSLA price test dataset")

p.title.text = 'LSTM forecast vs actual stock prices'
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha=0.3

In [ ]:
show(p)

## Kalman filter excercise

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from pykalman import KalmanFilter

In [ ]:
y_true = real_stock_price
y_pred = predicted_stock_price
mean_squared_error(y_true, y_pred)

In [ ]:
df = test_set
df.head()

In [ ]:
kf = KalmanFilter(transition_matrices = [1],
                  observation_matrices = [1],
                  initial_state_mean = df['Close'].values[0],
                  initial_state_covariance = 1,
                  observation_covariance=1,
                  transition_covariance=.01)

In [ ]:
state_means,_ = kf.filter(df[['Close']].values)
state_means = state_means.flatten()

In [ ]:
df['date'] = pd.to_datetime(df.Date)

mids = (df.Open + df.Close)/2
spans = abs(df.Close-df.Open)

inc = df.Close > df.Open
dec = df.Open > df.Close
w = 12*60*60*1000 # half day in ms

In [ ]:
output_notebook()

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p = figure(x_axis_type="datetime", tools=TOOLS, plot_width=1000, toolbar_location="left",y_axis_label = "Price",
          x_axis_label = "Date")

p.segment(df.date, df.High, df.date, df.Low, color="black")
p.rect(df.date[inc], mids[inc], w, spans[inc], fill_color='green', line_color="green")
p.rect(df.date[dec], mids[dec], w, spans[dec], fill_color='red', line_color="red")
p.line(df.date,state_means,line_width=1,line_color = 'blue',legend="Kalman filter")
p.line(df.date,real_stock_price.flatten(),line_width=1.4,line_color = 'black',legend="TSLA price test dataset")

p.title.text = 'Implementation of Kalman Filter Smoothing'
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha=0.3

In [ ]:
show(p)

In [ ]:
y_true = real_stock_price.flatten()
y_pred = state_means
mean_squared_error(y_true, y_pred)